In [1]:
import pandas as pd
import json
from generate_graph import get_propositions, generateEdges, createGraph, get_propositions_nosplit
from refine_graph import refine
from query_graph import QueryGraph
from tqdm import tqdm
tqdm.pandas()

In [2]:
df = pd.read_json('datasets/HotPotQA/hotpot_dev_distractor_v1.json')
# df.info()

## Experiment


### Graph creation and refinement section

To run the evaluation script

```
python hotpot_evaluate_v1.py <path_to_prediction> <path_to_gold>
```

Example:

```
python hotpot_evaluate_v1.py <sample_dev_pred.json> <hotpot_dev_fullwiki_v1.json>
```

In [26]:
context_list = df['context']

In [27]:
context_list[3:4]

3    [[Esma Sultan (daughter of Abdülaziz), [Esma S...
Name: context, dtype: object

In [28]:
propositions = []

for i, value in enumerate(tqdm(context_list[3:4])):
        for j, context in enumerate(value):
                index = 1
                if index < len(context):
                    words = context[1]
                    combined = " ".join(words)
                    get_propositions_nosplit(combined, propositions)

100%|██████████| 1/1 [00:45<00:00, 45.00s/it]


In [30]:
# Open the file in read mode
with open('propositions.txt', 'r') as file:
    # Read all lines and store them in a list
    propositions_from_file = [line.strip() for line in file]

print(propositions_from_file)

['Ed Wood is a 1994 American biographical period comedy-drama film.', 'Tim Burton directed and produced Ed Wood.', 'Johnny Depp stars as cult filmmaker Ed Wood in Ed Wood.', "The film concerns the period in Ed Wood's life when Ed Wood made his best-known films.", "The film also concerns Ed Wood's relationship with actor Bela Lugosi.", 'Martin Landau plays the role of Bela Lugosi in Ed Wood.', 'Sarah Jessica Parker is among the supporting cast of Ed Wood.', 'Patricia Arquette is among the supporting cast of Ed Wood.', 'Jeffrey Jones is among the supporting cast of Ed Wood.', 'Lisa Marie is among the supporting cast of Ed Wood.', 'Bill Murray is among the supporting cast of Ed Wood.', 'Scott Derrickson was born on July 16, 1966.', 'Scott Derrickson is an American director.', 'Scott Derrickson is a screenwriter.', 'Scott Derrickson is a producer.', 'Scott Derrickson lives in Los Angeles, California.', 'Scott Derrickson is best known for directing horror films.', "Scott Derrickson directed

In [18]:
list_of_edges = generateEdges(propositions_from_file)

  0%|          | 0/140 [00:00<?, ?it/s]

100%|██████████| 140/140 [05:42<00:00,  2.44s/it]


In [19]:
if createGraph(list_of_edges):
    print("Success")
else:
    print("Failed")

Success


In [ ]:
qg = QueryGraph()

if refine(qg, "person"):
    print("yes")
else:
    print("no")

### Q and A section

In [22]:
# def find_supporting_facts(question, answer, context):
#     supporting_facts = []
#     for passage in context:
#         title, sentences = passage
#         for i, sentence in enumerate(sentences):
#             if answer in sentence or any(word in sentence for word in question.split()):
#                 supporting_facts.append([title, i])
#     return supporting_facts

In [ ]:
# qg = QueryGraph()
# my_dict = {}

# # Define a function to apply to each row
# def print_qa(row):
    
#     question = row['question']
#     questionid = row['_id']
#     print("question:", question)
#     result = qg.answer_question(question)

#     if result is not None:
    
#         model_answer = result['result']

#         if "don't know the answer" in model_answer:
#             model_answer = ""
    
#         if len(row['answer']) > 0:
#             real_answer = row['answer']
#         else:
#             real_answer = "\"\""

#         if len(result['intermediate_steps']) > 0:
#             cypher_query = result['intermediate_steps'][0]['query']
#             context = result['intermediate_steps'][1]['context']
       
#             if(len(context) < 1):
#             #     result = refine_query(cypher_query[6:], question)
#             #     model_answer = result['result'] 
#                 model_answer = "\"\""
#                 my_dict.update({questionid: ""})
#             else:
#                 my_dict.update({questionid: model_answer})
                
#         with open("preds.txt", "a") as preds:
#             preds.write("question: " + question + "\n")
#             preds.write("real_answer: " + real_answer + "\n")
#             preds.write("model_answer: " + model_answer + "\n")  
#             preds.write("cypher_query: " + cypher_query + "\n")
#             preds.write("======================" + "\n")  
#         preds.close()

#     else: 
#         my_dict.update({questionid: ""})
        

In [ ]:
# my_dict = {}
# # Apply the function to each row
# df[0:1].progress_apply(print_qa, axis=1)
    
# with open("dev_fullwiki_pred.json", "a") as json_file:
#     json.dump(my_dict, json_file, indent=4)

  0%|          | 0/1 [00:00<?, ?it/s]

question: Were Scott Derrickson and Ed Wood of the same nationality?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p1:Person)-[r1:RELATED]-(e1:Entity), (p2:Person)-[r2:RELATED]-(e2:Entity)
WHERE toLower(p1.name) contains 'scott derrickson' AND toLower(p2.name) contains 'ed wood'
RETURN r1.description AS description1, r2.description AS description2, r1.metadata AS metadata1, r2.metadata AS metadata2

Full Context:
[{'description1': 'Scott Derrickson wrote the film Doctor Strange.', 'description2': 'Woodson is named after Ed Wood Sr.', 'metadata1': '{"summary": "Scott Derrickson wrote the film Doctor Strange with Jon Spaihts and C. Robert Cargill.", "generated_at": "2024-11-11 11:41:00.832541"}', 'metadata2': '{"summary": "Woodson, Woodson Lake, and Wood Hollow are the namesake for Ed Wood Sr.", "generated_at": "2024-11-11 11:41:00.832541"}'}, {'description1': 'Scott Derrickson wrote the film Doctor Strange.', 'description2': 'Wood Hollow is named after Ed 

100%|██████████| 1/1 [00:03<00:00,  3.51s/it]


> Finished chain.


In [3]:
question = df.iloc[1]['question']

qg = QueryGraph()
res = qg.test_query("Who is Shirley Temple?")

In [4]:
res

AIMessage(content="```cypher\nMATCH (e:Person)-[]-(entity:Entity)\nWHERE toLower(e.name) contains 'shirley temple'\nWITH e\nMATCH (e)-[rel:RELATED]-(relatedEntity:Entity)\nRETURN rel.description\n```", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 618, 'total_tokens': 669, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-c6d8d184-24fc-43b6-af33-8e25f8fb890e-0', usage_metadata={'input_tokens': 618, 'output_tokens': 51, 'total_tokens': 669})

In [ ]:
res2 = qg.answer_question("What government position was held by the woman who portrayed Corliss Archer in the film Kiss and Tell?")
res2['result']



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (role:Entity)-[]-(e:Person)-[]-(film:Entity)
WHERE toLower(role.name) = "corliss archer" AND toLower(film.name) = "kiss and tell"
WITH e
MATCH (e)-[rel:RELATED]-(p:Entity)
WHERE toLower(rel.description) CONTAINS 'government position' OR toLower(rel.description) CONTAINS 'position'
RETURN rel.description

Full Context:
[{'rel.description': 'Shirley Temple Black held the position of a diplomat.'}, {'rel.description': 'Shirley Temple Black held the position of Chief of Protocol of the United States.'}]

> Finished chain.


'Shirley Temple Black held the position of Chief of Protocol of the United States.'

In [11]:
question = df.iloc[0]['question']
res3 = qg.answer_question(question)
res3['result']



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (e1:Person)-[r:RELATED]-(e2:Person)
WHERE toLower(e1.name) = 'scott derrickson' AND toLower(e2.name) = 'ed wood'
WITH e1, e2, r
MATCH (e1)-[rel:RELATED]-(entity:Entity)
WHERE toLower(rel.description) CONTAINS 'nationality'
RETURN rel.description

Full Context:
[]

> Finished chain.


"I don't know the answer."

In [3]:
qg = QueryGraph()
res2 = qg.answer_question("What government position was held by the woman who portrayed Corliss Archer in the film Kiss and Tell?")
res2['result']



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (movie:Entity)-[r:RELATED]-(actress:Person)                                  
WHERE (toLower(r.metadata) CONTAINS 'kiss and tell') OR (toLower(r.description) CONTAINS 'kiss and tell')
WITH actress
MATCH (actress)-[r2:RELATED]-(position:Entity)
WHERE (toLower(r2.metadata) CONTAINS 'government') OR (toLower(r2.description) CONTAINS 'government')
RETURN position

Full Context:
[]

> Finished chain.


"I don't know the answer."

In [24]:
question = df.iloc[3]['question']
print(question)
answer = df.iloc[3]['answer']
print(answer)

Are the Laleli Mosque and Esma Sultan Mansion located in the same neighborhood?
no


In [25]:
context = df.iloc[3]['context']
print(context)

[['Esma Sultan (daughter of Abdülaziz)', ['Esma Sultan (21 March 1873 – 7 May 1899) was an Ottoman princess, the daughter of Sultan Abdülaziz and his wife Gevheri Kadın, herself the daughter of Salih Bey Svatnba.', ' She was the half-sister of Abdülmecid II, the last Caliph of the Muslim world.']], ['Djamaâ el Kebir', ['The Great Mosque of Algiers (Arabic: الجامع الكبير\u200e \u200e , "Jemaa Kebir") or “Djama’a al-Kebir” (meaning Great Mosque) is a mosque in Algiers, Algeria, located very close to Algiers Harbor.', ' An inscription on the minbar (منبر) or the pulpit testifies to fact that the mosque was built in 1097.', " It is also known by several other names such as Grand Mosque d'Alger, Djamaa al-Kebir, El Kebir Mosque and Jami Masjid.", ' It is one of the few remaining examples of Almoravid architecture.', ' It is the oldest mosque in Algiers and is said to be the oldest mosque in Algeria after Sidi Okba Mosque.', ' It was built under sultan Ali ibn Yusuf.', ' Its minaret dates fr

In [22]:
qg = QueryGraph()
res2 = qg.answer_question(question)
res2['result']



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (series:Entity)-[r:RELATED]-(companion:Entity) 
WHERE (toLower(r.description) CONTAINS 'science fantasy young adult series') OR (toLower(r.metadata) CONTAINS 'science fantasy young adult series')
WITH series, companion
MATCH (companion)-[r2:RELATED]-(worlds:Entity) 
WHERE (toLower(r2.description) CONTAINS 'enslaved worlds') OR (toLower(r2.metadata) CONTAINS 'enslaved worlds')
WITH series, companion, worlds
MATCH (companion)-[r3:RELATED]-(species:Entity) 
WHERE (toLower(r3.description) CONTAINS 'alien species') OR (toLower(r3.metadata) CONTAINS 'alien species')
RETURN series, companion, worlds, species

Full Context:
[]

> Finished chain.


"I don't know the answer."

## Graph refinement 

In [ ]:
qg = QueryGraph()

QUERY = """
MATCH (N) RETURN N
"""
res = qg._graph.query(QUERY)
res

In [3]:
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings

In [6]:
neo4j_url = "bolt://localhost:7687"
neo4j_user = "neo4j"
neo4j_password = "12345678"

Create embeddings

In [ ]:
# # Now we initialize from existing graph
# existing_graph = Neo4jVector.from_existing_graph(
#     embedding=OpenAIEmbeddings(model='text-embedding-3-small'),
#     url=neo4j_url,
#     username=neo4j_user,
#     password=neo4j_password,
#     index_name="person_index",
#     node_label="Person",
#     text_node_properties=["name"],
#     embedding_node_property="embedding",
# )

Refactor node labels

In [ ]:
# # cq = """
# # MATCH (en:Entity)
# # WHERE en.label = 'Person'
# # WITH collect(en) AS persons
# # CALL apoc.refactor.rename.label("Entity", "Person", persons)
# # YIELD batches, total, timeTaken, committedOperations
# # RETURN batches, total, timeTaken, committedOperations;
# # """
# # res = qg._graph.query(cq)

Create projection

In [ ]:
cq = """
MATCH (p:Person)
RETURN gds.graph.project(
  'personproj',
  p,
  null,
  {
    sourceNodeProperties: p { .embedding },
    targetNodeProperties: {}
  }
)
"""

res = qg._graph.query(cq)

Use KNN to measure node similarities

In [236]:
cq = """
CALL gds.knn.stream('projection2', {
topK: 1,
nodeProperties: ['embedding'],
// The following parameters are set to produce a deterministic result
randomSeed: 1337,
concurrency: 1,
sampleRate: 1.0,
deltaThreshold: 0.0
})
YIELD node1, node2, similarity
WHERE similarity > .90
RETURN gds.util.asNode(node1).name AS Person1, gds.util.asNode(node2).name AS Person2, similarity
ORDER BY similarity DESCENDING, Person1, Person2
"""

res = qg._graph.query(cq)
res

[{'Person1': 'Ed Wood',
  'Person2': 'Ed Wood Sr',
  'similarity': 0.9346900582313538},
 {'Person1': 'Ed Wood Sr',
  'Person2': 'Ed Wood',
  'similarity': 0.9346900582313538}]

In [237]:
for i in res:
    for key, value in i.items():
        print(f"{key}: {value}")

Person1: Ed Wood
Person2: Ed Wood Sr
similarity: 0.9346900582313538
Person1: Ed Wood Sr
Person2: Ed Wood
similarity: 0.9346900582313538


In [238]:
def are_dictionaries_equivalent(d1, d2):
    # Check if both have the same keys
    if d1.keys() != d2.keys():
        return False

    # Check if Person1 and Person2 are swapped but equivalent
    if (d1['Person1'] == d2['Person2'] and d1['Person2'] == d2['Person1'] and d1['similarity'] == d2['similarity']):
        return True
    
    return False


In [239]:
newlist = res

In [240]:
for idx, val in enumerate(res):   
    for idx2, val2 in enumerate(newlist):
        if val != val2:
            if are_dictionaries_equivalent(val, val2):
                newlist.pop(idx2)


In [241]:
unique = []

for value in newlist:
    if value not in unique:
        unique.append(value)

unique

[{'Person1': 'Ed Wood',
  'Person2': 'Ed Wood Sr',
  'similarity': 0.9346900582313538}]

In [233]:
node_name = "Shirley Temple"

query = f"""
MATCH (n:Person {{name: '{node_name}'}})
RETURN n
"""
print(query)

res = qg._graph.query(query)
res



MATCH (n:Person {name: 'Shirley Temple'})
RETURN n



[{'n': {'name': 'Shirley Temple',
   'embedding': [-0.0038625726010650396,
    -0.004033572040498257,
    -0.052681200206279755,
    -0.01091713272035122,
    -0.04026195406913757,
    0.00812749657779932,
    0.022437792271375656,
    0.07430087774991989,
    -0.05445154756307602,
    -0.039457254111766815,
    0.020882032811641693,
    0.001297582988627255,
    0.008328672498464584,
    -0.025549309328198433,
    -0.031302932649850845,
    0.01650981605052948,
    0.009629608131945133,
    0.05906517431139946,
    -0.006585150025784969,
    0.039189018309116364,
    0.016845108941197395,
    0.003584279678761959,
    -0.00032439574715681374,
    -0.0011844217078760266,
    0.01050807535648346,
    0.05455883964896202,
    -0.04254194721579552,
    -0.06695125997066498,
    0.024382490664720535,
    -0.030900580808520317,
    0.028835177421569824,
    -0.043400295078754425,
    -0.0028013710398226976,
    -0.008898669853806496,
    -0.004070454277098179,
    0.007135030347853899,
    

Merge similar nodes (Entity resolution)

In [242]:
for idx, val in enumerate(unique):
    print(val)
    p1 = val['Person1']
    p2 = val['Person2']    
    cq = f"""
    MATCH (a1:Person {{name: '{p1}'}}), (a2:Person {{name: '{p2}'}})
    WITH head(collect([a1,a2])) as nodes
    CALL apoc.refactor.mergeNodes(nodes,{{
    properties:"discard",
    mergeRels:true
    }})
    YIELD node
    RETURN node;
    """
    res = qg._graph.query(cq)
    res

{'Person1': 'Ed Wood', 'Person2': 'Ed Wood Sr', 'similarity': 0.9346900582313538}
